<a href="https://colab.research.google.com/github/Samin-Sadaf7/Book-of-LLMs/blob/main/HandsOnLLM_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2
#!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

#Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    )
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#Create a pipeline
pipe =pipeline(
    "text-generation",
    model = model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda


In [3]:
#Prompt
messages = [
    {
        "role":"user",
        "content":"Create a funny joke about chickens."
    }
]
#Generate the output
output = pipe(messages)
print(output[0]["generated_text"])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Why did the chicken join the band? Because it had the drumsticks!


In [4]:
#Apply Prompt template
prompt = pipe.tokenizer.apply_chat_template(
    messages,
    tokenize=False
)
print(prompt)

<|user|>
Create a funny joke about chickens.<|end|>
<|endoftext|>


In [5]:
#Using high temperature
for _ in range(10):
  output = pipe(messages, do_sample=True, temperature=1)
  print(output[0]["generated_text"])

 Why did the chicken join a band?


Because it wanted to learn how to cluck-et well!
 Why don't chickens ever play hide and seek? Because good luck hiding when they always end up pecking in the most obvious places!
 Why did the chicken go to the computer? Because it wanted to find out the "egg-cellent" code!
 Why did the chicken join the book club?


Because it wanted to learn more about the deep "roe"s of literature!
 Why did the chicken join a band? Because it heard the punchline to the joke "why did the scarecrow win an award? Because he was outstanding in his field." The chicken didn't want to miss out on the fun anymore!
 Why do chickens make terrible singers? Because they always give "cock-a-doodle-doodle" instead of notes!
 Why did the chicken cross the playground?
 Why don't chickens use computers? They don't know how to click!


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 Why do chickens like running around the yard?

Because they're always looking for a golden egg!
 Why do chickens never play hide and seek? Because good luck hiding behind a chicken when you know the rooster always 'crows' it!


In [6]:
#Using high top_p
for _ in range(5):
  output = pipe(messages, do_sample=True, top_p=1)
  print(output[0]['generated_text'])

 Why did the chicken join a band? Because it had the drumsticks!
 Why do chickens never gossip? Because their teeth are too pointed!
 Why don't chickens play basketball? Because they're afraid of the hoop and can't deal with the free throws after a lay-up.
 Why don't chickens use computers?
 Why don't chickens use the internet?


In [7]:
#Using top_k (at most k tokens will consider at all tokens)
for _ in range(5):
  output = pipe(messages, do_sample=True, top_k=10)
  print(output[0]['generated_text'])

 How do chickens always know what to wear? They never wear a hat, because they always think they're in a pecking good fashion!
 Why don't chickens use social media? They can't handle it, there's too much eggstra content!
 Why do chickens always cross the road? 

Because they prefer to take their eggs to a new adventure, and they've heard it's the best place to find a good cluck-berry bush!
 Why don’t chickens use computers? Because they can't type their clucks without making a mess!
 Why don't chickens use computers? Because they're scared they might get pecked by the mouse!


##Prompt Engineering

In [8]:
# Text to summarize which we stole from https://jalammar.github.io/illustrated-transformer/ ;)
text = """In the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neural Machine Translation model in specific tasks. The biggest benefit, however, comes from how The Transformer lends itself to parallelization. It is in fact Google Cloud’s recommendation to use The Transformer as a reference model to use their Cloud TPU offering. So let’s try to break the model apart and look at how it functions.
The Transformer was proposed in the paper Attention is All You Need. A TensorFlow implementation of it is available as a part of the Tensor2Tensor package. Harvard’s NLP group created a guide annotating the paper with PyTorch implementation. In this post, we will attempt to oversimplify things a bit and introduce the concepts one by one to hopefully make it easier to understand to people without in-depth knowledge of the subject matter.
Let’s begin by looking at the model as a single black box. In a machine translation application, it would take a sentence in one language, and output its translation in another.
Popping open that Optimus Prime goodness, we see an encoding component, a decoding component, and connections between them.
The encoding component is a stack of encoders (the paper stacks six of them on top of each other – there’s nothing magical about the number six, one can definitely experiment with other arrangements). The decoding component is a stack of decoders of the same number.
The encoders are all identical in structure (yet they do not share weights). Each one is broken down into two sub-layers:
The encoder’s inputs first flow through a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. We’ll look closer at self-attention later in the post.
The outputs of the self-attention layer are fed to a feed-forward neural network. The exact same feed-forward network is independently applied to each position.
The decoder has both those layers, but between them is an attention layer that helps the decoder focus on relevant parts of the input sentence (similar what attention does in seq2seq models).
Now that we’ve seen the major components of the model, let’s start to look at the various vectors/tensors and how they flow between these components to turn the input of a trained model into an output.
As is the case in NLP applications in general, we begin by turning each input word into a vector using an embedding algorithm.
Each word is embedded into a vector of size 512. We'll represent those vectors with these simple boxes.
The embedding only happens in the bottom-most encoder. The abstraction that is common to all the encoders is that they receive a list of vectors each of the size 512 – In the bottom encoder that would be the word embeddings, but in other encoders, it would be the output of the encoder that’s directly below. The size of this list is hyperparameter we can set – basically it would be the length of the longest sentence in our training dataset.
After embedding the words in our input sequence, each of them flows through each of the two layers of the encoder.
Here we begin to see one key property of the Transformer, which is that the word in each position flows through its own path in the encoder. There are dependencies between these paths in the self-attention layer. The feed-forward layer does not have those dependencies, however, and thus the various paths can be executed in parallel while flowing through the feed-forward layer.
Next, we’ll switch up the example to a shorter sentence and we’ll look at what happens in each sub-layer of the encoder.
Now We’re Encoding!
As we’ve mentioned already, an encoder receives a list of vectors as input. It processes this list by passing these vectors into a ‘self-attention’ layer, then into a feed-forward neural network, then sends out the output upwards to the next encoder.
"""

In [9]:
#Prompt Components
persona ="You are an expert in Large Language Models. You exel at breaking down complex paper into digestible summaries.\n"
instruction="Summarize the key findings of the paper provided.\n"
context = "Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.\n"
data_format="Create a bullet-point summary that outlines the method. Follow this up with concise paragraph that encapsulates the main results.\n"
audience="The summary designed for busy researchers that quickly need to grasp the newest trend in Large Language Models.\n"
tone="The tone should be professional and clear.\n"
data = f"Text to summarize: {text}"
query = persona +instruction + context+ data_format +audience + tone+ data

In [10]:
messages =[
    {
        "role":"user",
        "content": query
    }
]
print(pipe.tokenizer.apply_chat_template(messages, tokenize=False))

<|user|>
You are an expert in Large Language Models. You exel at breaking down complex paper into digestible summaries.
Summarize the key findings of the paper provided.
Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.
Create a bullet-point summary that outlines the method. Follow this up with concise paragraph that encapsulates the main results.
The summary designed for busy researchers that quickly need to grasp the newest trend in Large Language Models.
The tone should be professional and clear.
Text to summarize: In the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neural Machi

In [11]:
output= pipe(messages)
print(output[0]['generated_text'])

 **Method:**

- The Transformer model uses attention mechanisms to improve the training speed of deep learning models.
- It consists of an encoding component and a decoding component, with six encoders and six decoders stacked on top of each other.
- Encoders have two sub-layers: a self-attention layer and a feed-forward neural network.
- Decoders have the same sub-layers as encoders, with an additional attention layer.
- Input words are converted into vectors using an embedding algorithm.
- The size of the list of vectors is a hyperparameter that can be set based on the length of the longest sentence in the training dataset.
- The word in each position flows through its own path in the encoder, with dependencies in the self-attention layer and parallel execution in the feed-forward layer.

**Results:**

The Transformer model has shown to outperform the Google Neural Machine Translation model in specific tasks. Its main benefit lies in its ability to parallelize the training process, m

In [12]:
#Using a single example of using the made-up word in a sentence
one_shot_prompt= [
    {
        "role":"user",
        "content":"A 'Gigamuru' is a type of Japanese Musical Instrument. An example of a sentence that uses the word Gigamuru is: "
    },
    {
        "role":"assistant",
        "content":"I have a Gigamuru that my uncle gave me as a gift. I love to play it at home."
    },{
        "role": "user",
        "content":"To 'screeg' something is to swing a swort at it. An example of a sentence that uses the word screeg is:"
    }
]

In [13]:
print(pipe.tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

<|user|>
A 'Gigamuru' is a type of Japanese Musical Instrument. An example of a sentence that uses the word Gigamuru is: <|end|>
<|assistant|>
I have a Gigamuru that my uncle gave me as a gift. I love to play it at home.<|end|>
<|user|>
To 'screeg' something is to swing a swort at it. An example of a sentence that uses the word screeg is:<|end|>
<|endoftext|>


In [14]:
#Generate the output
output= pipe(one_shot_prompt)
print(output[0]['generated_text'])

 During the game, the player screeg the swort at the opposing team's goalkeeper, hoping to score a point.


##Prompt Chaining
Take the output of one prompt and use it for the next, there by creating a continuous chain of interactions that solves the our problem

In [16]:
#Chaining Prompt
#Create name and slogan for a product
product_prompt = [
    {
        "role":"user",
        "content":"Create a name and slogan for a chatbot that leverages LLMs."
    }
]
outputs = pipe(product_prompt)
product_descriptions = outputs[0]['generated_text']
print(product_descriptions)

 Name: ChatSage
Slogan: "Unleashing the power of AI to enhance your conversations."


In [18]:
#Based on name and slogan for a product, generate a sales pitch
sales_prompt =[
    {
        "role":"user",
        "content":f"Generate a very short sales pitch for the following product: '{product_descriptions}'"
    }
]
outputs = pipe(sales_prompt)
sales_pitch = outputs[0]['generated_text']
print(sales_pitch)

 Introducing ChatSage, the revolutionary AI-powered tool designed to elevate your conversations to new heights. With our cutting-edge technology, we unleash the power of AI to enhance your interactions, making every conversation more engaging, insightful, and meaningful. Experience the future of communication with ChatSage today!


Use cases:
1. Response Validation
2. Parallel Prompts
3. Writing Stories

##Reasoning with Generative Model



*   Chain of Thoughts



In [19]:
#Answering with chain of thought
cot_prompt = [
    {
        "role":"user",
        "content":"Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"
    },{
        "role":"assitant",
        "content":"Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."
    },{
        "role":"user",
        "content":"The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"
    }
]
#Generate output
outputs = pipe(cot_prompt)
print(outputs[0]['generated_text'])

 The cafeteria had 23 apples.
They used 20 apples to make lunch, so they have 23 - 20 = 3 apples left.
They bought 6 more apples, so they now have 3 + 6 = 9 apples.



* Zero Shot chain of thought

In [21]:
#Zero shot chain of though prompting
zero_shot_cot_prompt = [
    {
        "role":"user",
        "content":"The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? Let's think step by step."
    }
]
outputs= pipe(zero_shot_cot_prompt)
print(outputs[0]['generated_text'])

 Step 1: The cafeteria starts with 23 apples.

Step 2: They use 20 apples to make lunch. To find out how many apples are left, we subtract the number of apples used from the initial amount:
23 apples - 20 apples = 3 apples

Step 3: The cafeteria then buys 6 more apples. To find the new total, we add the apples bought to the remaining apples:
3 apples + 6 apples = 9 apples

So, after using some apples for lunch and buying more, the cafeteria has 9 apples.


* Self-Consistenct:

  Same prompt can generate different results. It can be affected by temperature or top_p values. Self-consistency method asks same prompt to LLM multiple times. The diversity of the answers depend on temperature and top_p values. It takes the majority result as the output.

* Tree of thought

In [22]:
zero_shot_tot_prompt = [
    {
        "role":"user",
        "content":"Imagine three different experts are answering this questions. All experts will write down 1 step of their thinking \
        , then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they're wrong at any point then they leave\
        . The question is 'The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?' Make sure to discuss the results."
    }
]
outputs = pipe(zero_shot_tot_prompt)
print(outputs[0]['generated_text'])

 Expert 1:
Step 1: Start with the initial number of apples, which is 23.

Expert 2:
Step 1: Subtract the number of apples used for lunch, which is 20.
Result: 23 - 20 = 3 apples remaining.

Expert 3:
Step 1: Add the number of apples bought, which is 6.
Result: 3 + 6 = 9 apples remaining.

Discussion:
All three experts arrived at the same answer, which is 9 apples remaining in the cafeteria. This result is obtained by subtracting the apples used for lunch from the initial number and then adding the apples bought. The experts' thought processes were consistent and accurate, leading to the same correct answer.


* Output Verification:
Controlling the output generated by LLM:
  - Examples
  - Grammar
  - Fine-tuning

In [24]:
#Zero Shot Learning: providing no example
zeroshot_prompt = [{
    "role":"user",
    "content": "Create a character profile for an RPG game in JSON format"
}]
outputs = pipe(zeroshot_prompt)
print(outputs[0]['generated_text'])

 ```json

{

  "character_name": "Aelar the Swift",

  "class": "Rogue",

  "level": 1,

  "attributes": {

    "strength": 10,

    "dexterity": 18,

    "constitution": 12,

    "intelligence": 14,

    "wisdom": 13,

    "charisma": 11

  },

  "skills": {

    "stealth": 15,

    "lockpicking": 14,

    "acrobatics": 13,

    "perception": 12

  },

  "equipment": {

    "weapon": "Dagger",

    "armor": "Leather Armor",

    "accessories": ["Gloves of Dexterity", "Boots of Speed"]

  },

  "background": "Aelar was born into a family of thieves. He learned the art of stealth and deception from a young age. He is always looking for the next big heist and is not afraid to take risks."

}

```


In [25]:
#One shot Learning : Providing an example of the output structure
one_shot_template = """ Create a short character profile for an RPG game. Make sure to only use this format:
{
  "description":"A SHORT DESCRIPTION",
  "name": "THE CHARACTER'S NAME",
  "armor" : "ONE PIECE OF ARMOR",
  "weapon": "ONE OR MORE WEAPONS"
}
"""
one_shot_prompt = [
    {
        "role":"user",
        "content" : one_shot_template
    }
]
#Generate the output
outputs =  pipe(one_shot_prompt)
print(outputs[0]['generated_text'])

 {
  "description": "A cunning rogue with a mysterious past, skilled in stealth and deception.",
  "name": "Shadowcloak",
  "armor": "Leather Hood",
  "weapon": "Dagger"
}


* Grammmar: Constrained Sampling

In [26]:
import gc
import torch
del model, tokenizer, pipe

#Flush memory
gc.collect()
torch.cuda.empty_cache()

In [28]:
!pip install llama-cpp-python

  Using cached llama_cpp_python-0.3.5.tar.gz (64.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.5-cp310-cp310-linux_x86_64.whl size=3471437 sha256=efb0cbad3104c1dd07ae2eb80000d3af81297be6c4f303bfa009a8b97939dcdd
  Stored in directory: /root/.cache/pip/wheels/7e/ae/c3/34a10eba5fc4e75df17818312b6a1ee29eb90f13a12c524714
Successfully built llama-cpp-python


In [29]:
from llama_cpp.llama import Llama

#Load Phi-3
llm = Llama.from_pretrained(
    repo_id = "microsoft/Phi-3-mini-4k-instruct-gguf",
    filename="*fp16.gguf",
    n_gpu_layers = -1,
    n_ctx = 2048,
    verbose = False
)

Phi-3-mini-4k-instruct-fp16.gguf:   0%|          | 0.00/7.64G [00:00<?, ?B/s]

llama_new_context_with_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [31]:
#Generate ouput
output = llm.create_chat_completion(
    messages =[
        {
            "role":"user",
            "content":"Create a warrior for an RPG in JSON format."
        },
    ],
        response_format = {
            "type": "json_object"
        },
        temperature= 0
)['choices'][0]['message']['content']

In [32]:
import json
#Format as json
json_output = json.dumps(json.loads(output), indent=4)
print(json_output)

{
    "warrior": {
        "name": "Eldric Stormbringer",
        "class": "Warrior",
        "level": 5,
        "attributes": {
            "strength": 18,
            "dexterity": 10,
            "constitution": 16,
            "intelligence": 8,
            "wisdom": 10,
            "charisma": 12
        },
        "skills": [
            {
                "name": "Martial Arts",
                "proficiency": 18
            },
            {
                "name": "Heavy Armor",
                "proficiency": 16
            },
            {
                "name": "Swordsmanship",
                "proficiency": 17
            },
            {
                "name": "Shield Mastery",
                "proficiency": 15
            },
            {
                "name": "Survival",
                "proficiency": 12
            }
        ],
        "equipment": [
            {
                "name": "Iron Sword",
                "type": "Weapon",
                "damage": 12,
    